# LaWaF demos

# SCDM-k Demo (electron)

Downfolding the electron Wannier model  from Mn3d+O 2p to Mn 3d only.

In [3]:
from lawaf import W90Downfolder
import numpy as np
from ipywidgets import widgets

@widgets.interact_manual(band=["eg", "t2g", "unspecified"], mu=(-1, 13, 0.1),sigma=(0.0, 13, 0.1), nk=(2, 8, 1), nwann=(1, 14,1), weight_func=[ "Gauss", "Fermi", "unity"])
def Downfold_w90_hamiltonian_of_SMO(band="eg", mu=9.0, sigma=3.0, weight_func="gauss", use_proj=False, nk=4, nwann=4
 ):
    # Read From Wannier90 output
    model = W90Downfolder(folder='../../example/Wannier90/SMO_wannier',
                          prefix='abinito_w90_down')

    if band=="eg":
        nwann=2
        anchors={(0, 0, 0): (12,13)}
    elif band=="t2g":
        nwann=3
        anchors={(0, 0, 0): (9, 10,11)}
    elif band=="unspecified":
        nwann=nwann
        anchors=None
    # Downfold the band structure.
    model.downfold(method="scdmk",
        kmesh=(nk, nk, nk),
        nwann=nwann,
        weight_func=weight_func,
        mu=mu,
        sigma=sigma,
        selected_basis=None,
        anchors=anchors,
        use_proj=use_proj,
        write_hr_nc='Downfolded_hr.nc',
        write_hr_txt='Downfolded_hr.txt' )

    # Plot the band structure.
    model.plot_band_fitting(
                          kvectors=np.array([[0, 0, 0], [0.5, 0, 0],
                                             [0.5, 0.5, 0], [0, 0, 0],
                                             [.5, .5, .5]]),
                          knames=['$\Gamma$', 'X', 'M', '$\Gamma$', 'R'],
                          supercell_matrix=None,
                          npoints=100,
                          efermi=None,
                          erange=None,
                          fullband_color='blue',
                          downfolded_band_color='green',
                          marker='o',
                          ax=None,
                          savefig='Downfolded_band.png',
                          show=True)
 

interactive(children=(Dropdown(description='band', options=('eg', 't2g', 'unspecified'), value='eg'), FloatSli…

# Projected Wannier function Demo (electron)

In [10]:
from lawaf import W90Downfolder
import numpy as np
from ipywidgets import widgets

@widgets.interact_manual(projectors=[ "input", "eg", "t2g",], mu=(-1, 13, 0.1),sigma=(0.0, 13, 0.1), nk=(2, 8, 1), nwann=(1, 14,1), weight_func=["unity", "Gauss", "Fermi"])
def Downfold_w90_hamiltonian_of_SMO(projectors="eg", input_projectors="1 2 4", mu=9.0, sigma=3.0, weight_func="gauss",  nk=4
 ):
    # Read From Wannier90 output
    model = W90Downfolder(folder='../../example/Wannier90/SMO_wannier',
                          prefix='abinito_w90_down')
    anchors=None
    band=projectors
    if band=="eg":
        nwann=2
        #anchors={(0, 0, 0): (12,13)}
        selected_basis=[0, 3]
    elif band=="t2g":
        nwann=3
        #anchors={(0, 0, 0): (9, 10,11)}
        selected_basis=[1,2,4]
    elif band=="input":
        anchors=None
        selected_basis=tuple(int(x) for x in input_projectors.split())
    nwann=len(selected_basis)       
    # Downfold the band structure.
    model.downfold(method="projected",
        kmesh=(nk, nk, nk),
        nwann=nwann,
        weight_func=weight_func,
        mu=mu,
        sigma=sigma,
        selected_basis=selected_basis,
        anchors=anchors,
        use_proj=False,
        write_hr_nc='Downfolded_hr.nc',
        write_hr_txt='Downfolded_hr.txt', 
                  )

    # Plot the band structure.
    model.plot_band_fitting(
                          kvectors=np.array([[0, 0, 0], [0.5, 0, 0],
                                             [0.5, 0.5, 0], [0, 0, 0],
                                             [.5, .5, .5]]),
                          knames=['$\Gamma$', 'X', 'M', '$\Gamma$', 'R'],
                          supercell_matrix=None,
                          npoints=100,
                          efermi=None,
                          erange=None,
                          fullband_color='blue',
                          downfolded_band_color='green',
                          marker='o',
                          ax=None,
                          savefig='Downfolded_band.png',
                          show=True)

interactive(children=(Dropdown(description='projectors', index=1, options=('input', 'eg', 't2g'), value='eg'),…

# Lattice Wannier function (SCDM-k): example of BaTiO3. Without LO-TO splitting.

In [7]:
from lawaf import W90Downfolder
from lawaf.scdm import PhonopyDownfolder
import numpy as np
from ipywidgets import widgets
import matplotlib.pyplot as plt


@widgets.interact_manual( mu=(-300, 900, 1),sigma=(0.0, 900, 0.1),nk=(2, 8, 1), nwann=(1, 15,1), weight_func=["Gauss", "Fermi","unity"])
def Downfold_phonon_of_BaTiO3(anchor_k=["Gamma", "X", "M", "R"], phonon="0  1 2", mu=-200, sigma=100, 
                              weight_func="gauss", use_proj=True, nk=4, nwann=3):
    fname="../../example/Phonopy/BaTiO3/nodip/phonopy_params.yaml"
    downfolder = PhonopyDownfolder(phonopy_yaml=fname, mode="DM")
    kdict={"Gamma":(0,0,0), "X":(.5, 0, 0), "M":(.5, .5, 0), "R":(.5,.5,.5)}
    phonon=tuple(int(x) for x in phonon.split())
    anchors={kdict[anchor_k]: phonon}
    # Downfold the band structure.
    downfolder.downfold(method="scdmk",
        kmesh=(nk, nk, nk),
        nwann=nwann,
        weight_func=weight_func,
        mu=mu,
        sigma=sigma,
        selected_basis=None,
        anchors=anchors,
        use_proj=use_proj,
        write_hr_nc='Downfolded_hr.nc',
        write_hr_txt='Downfolded_hr.txt' )

    ax = downfolder.plot_band_fitting(kvectors=np.array([[0., 0., 0.],
                                                     [0.5, 0.0, 0.],
                                                     [0.5, 0.5, 0.0],
                                                     [0.5, 0.5, 0.5],
                                                     [0.5, 0.0, 0.0],
                                                     [0.0, 0.0, 0],
                                                     [0.5, 0.5, 0.5]
                                                     ]), 
                                                     npoints=300,
                                                     unit_factor=15.6*33.6,
                                                     ylabel="Frequency (cm^-1)",
                                                     evals_to_freq=True,
                                  knames=['$\\Gamma$', 'X', 'M', 'R', 'X', '$\\Gamma$', "R"], show=False)
    plt.savefig('LWF_BTO.pdf')
    plt.show()


 

interactive(children=(Dropdown(description='anchor_k', options=('Gamma', 'X', 'M', 'R'), value='Gamma'), Text(…

# Projected Wannier function, example of BaTiO3. 

In [5]:
from lawaf import W90Downfolder
from lawaf.scdm import PhonopyDownfolder
import numpy as np
from ipywidgets import widgets
import matplotlib.pyplot as plt


@widgets.interact_manual( mu=(-300, 900, 1),sigma=(0.0, 900, 0.1),nk=(2, 8, 1), nwann=(1, 15,1), weight_func=["unity", "Gauss", "Fermi"])
def Downfold_phonon_of_BaTiO3(anchor_k=["Gamma", "X", "M", "R"], phonon="0  1 2", mu=-200, sigma=100, 
                              weight_func="gauss", use_proj=True, nk=4, nwann=3):
    fname="../../example/Phonopy/BaTiO3/nodip/phonopy_params.yaml"
    downfolder = PhonopyDownfolder(phonopy_yaml=fname, mode="DM")
    kdict={"Gamma":(0,0,0), "X":(.5, 0, 0), "M":(.5, .5, 0), "R":(.5,.5,.5)}
    phonon=tuple(int(x) for x in phonon.split())
    anchors={kdict[anchor_k]: phonon}
    # Downfold the band structure.
    downfolder.downfold(method="projected",
        kmesh=(nk, nk, nk),
        nwann=nwann,
        weight_func=weight_func,
        mu=mu,
        sigma=sigma,
        selected_basis=None,
        anchors=anchors,
        use_proj=use_proj,
        write_hr_nc='Downfolded_hr.nc',
        write_hr_txt='Downfolded_hr.txt' )

    ax = downfolder.plot_band_fitting(kvectors=np.array([[0., 0., 0.],
                                                     [0.5, 0.0, 0.],
                                                     [0.5, 0.5, 0.0],
                                                     [0.5, 0.5, 0.5],
                                                     [0.5, 0.0, 0.0],
                                                     [0.0, 0.0, 0],
                                                     [0.5, 0.5, 0.5]
                                                     ]), 
                                                     npoints=300,
                                                     unit_factor=15.6*33.6,
                                                     ylabel="Frequency (cm^-1)",
                                                     evals_to_freq=True,
                                  knames=['$\\Gamma$', 'X', 'M', 'R', 'X', '$\\Gamma$', "R"], show=False)
    plt.savefig('LWF_BTO.pdf')
    plt.show()

interactive(children=(Dropdown(description='anchor_k', options=('Gamma', 'X', 'M', 'R'), value='Gamma'), Text(…